# Aufgabe 1 - k-NN Klassifikation

## Teilaufgabe a)

## Teilaufgabe b)

## Teilaufgabe c)

In [71]:
import numpy as np
import pandas as pd

In [72]:
def dist(a,b):
    return np.linalg.norm(a-b)

def makeSortedDistanceList(samplePoint, trainingData, labels):
    distanceList = []
    counter = 0
    for j in trainingData:
        distance = dist(samplePoint, j)
        distanceList.append((distance, labels[counter]))
        counter += 1
    distanceList = np.asarray(distanceList)
    sortedDistanceList = distanceList[distanceList[:,0].argsort()]
    return sortedDistanceList

class KNN:
    '''KNN Classifier.

    Attributes
    ----------
    k : int
        Number of neighbors to consider.
    '''
    def __init__(self, k):
        '''Initialization.
        Parameters are stored as member variables/attributes.
        
        Parameters
        ----------
        k : int
            Number of neighbors to consider.
        '''
        self.k = k

    def fit(self, X, y):
        '''Fit routine.
        Training data is stored within object.
        
        Parameters
        ----------
        X : numpy.array, shape=(n_samples, n_attributes)
            Training data.
        y : numpy.array shape=(n_samples)
            Training labels.
        '''
        self.training_data = X
        self.training_labels = y

    def predict(self, X):
        '''Prediction routine.
        Predict class association of each sample of X.
        
        Parameters
        ----------
        X : numpy.array, shape=(n_samples, n_attributes)
            Data to classify.
        
        Returns
        -------
        prediction : numpy.array, shape=(n_samples)
            Predictions, containing the predicted label of each sample.
        '''
        
        # First we need to find a few parameters
        n_attributes, n_samples = X.shape # for whatever reason no parentheses after shape...
        n_training_data = np.size(self.training_data, axis = 1) # this means finding the amount of columns
        
        prediction = []
        
        for i in X: # for i in sampleData
            distanceList = []
            kNearestNeighbors = []
            back = 0
            sig = 0
            counter = 0
            sortedDistanceList = makeSortedDistanceList(samplePoint = i, trainingData = self.training_data, labels = self.training_labels)
            kNearestNeighbors = sortedDistanceList[:self.k]
            for l in range(0, self.k):
                if kNearestNeighbors[l,1] == 0:
                    back += 1
                else:
                    sig +=1
            if back > sig:
                prediction.append(0)
            elif back < sig:
                prediction.append(1)
            else:
                prediction.append(float('NaN'))
        return prediction

## Teilaufgabe d)

In [73]:
hdf = pd.read_hdf('NeutrinoMC.hdf5', key = 'Signal')
hits = hdf.NumberOfHits
x = hdf.x
y = hdf.y

# Dont know how to properly loop over the three arrays, they arent changed in the code below...?
#temp_array = [hits, x, y]
#for count in temp_array: 
    #count = np.asarray(count)
    #count = count[~np.isnan(count)] # ~ means not

hits = np.asarray(hits)
hits = hits[~np.isnan(hits)] # ~ means not 
x = np.asarray(x)
x = x[~np.isnan(x)]
y = np.asarray(y)
y = y[~np.isnan(y)]

matrix = np.matrix([hits, x, y])
print(matrix[:,0:3]) # Look at the first three columns to see if it worked properly

[[ 26.          38.          87.        ]
 [  6.69494405   7.74364571   7.41714979]
 [  2.20707621   3.57920725   2.96454426]]


In [74]:
trainingDataSignal = matrix[:,0:5000]
testDataSignal = matrix[:,5000:15000]
testDataSignal = np.transpose(testDataSignal)

In [75]:
hdf = pd.read_hdf('NeutrinoMC.hdf5', key = 'Background')
hits = hdf.NumberOfHits
x = hdf.x
y = hdf.y

hits = np.asarray(hits)
hits = hits[~np.isnan(hits)]
x = np.asarray(x)
x = x[~np.isnan(x)]
y = np.asarray(y)
y = y[~np.isnan(y)]

matrix = np.matrix([hits, x, y])
testDataBackground = matrix[:,0:20000]
testDataBackground = np.transpose(testDataBackground)
trainingDataBackground = matrix[:,20000:25000]

trainingData = np.concatenate((trainingDataBackground, trainingDataSignal), axis=1)
trainingData = np.transpose(trainingData)

y1 = np.zeros(5000)
y2 = np.ones(5000)
labels = np.concatenate((y1,y2)) # Lets say 0 is background and 1 is signal

In [76]:
knn = KNN(10)
knn.fit(trainingData, labels)
predictionS = knn.predict(testDataSignal[0:20])
predictionB = knn.predict(testDataBackground[0:20])

In [77]:
predictionS = np.asarray(predictionS)
tp = np.count_nonzero(predictionS == 1)
fp = np.count_nonzero(predictionS == 0)
tn = np.count_nonzero(predictionB == 1)
fn = np.count_nonzero(predictionB == 0)
Reinheit = tp/(tp+fp)
Effizienz = tp/(tp+fn)
Signifikanz = tp/(np.sqrt(tp+fp))
print('Reinheit =', Reinheit)
print('Effizienz =', Effizienz)
print('Signifikanz =', Signifikanz)

Reinheit = 1.0
Effizienz = 1.0
Signifikanz = 4.24264068712


In [83]:
% timeit predictionS = knn.predict(testDataSignal[0:1])

1 loop, best of 3: 183 ms per loop
